# <center>Machine Learning Project</center>

** **
## <center>*03.7 - Neural Networks*</center>

** **

The members of the `team` are:
- Ana Farinha - 20211514
- Francisco Capontes - 20211692
- Sofia Gomes - 20240848
- Rui Lourenço - 2021639



In [1]:
# Import libraries
import pandas as pd
import numpy as np

#make the split here
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold
import time


from utils import *
from utils_dicts import *

## <span style="color:salmon"> 1. Import Dataset </span> 

In [2]:
# Import dataset
df = pd.read_csv('preprocessed_data/train_data.csv', index_col="Claim Identifier")
df

,Accident Date,Age at Injury,Average Weekly Wage,Birth Year,IME-4 Count,Number of Dependents,Known Accident Date,Known Assembly Date,Known C-2 Date,Known C-3 Date,...,Zip_Code_Hash_13,Zip_Code_Hash_14,Zip_Code_Hash_15,Zip_Code_Hash_16,Zip_Code_Hash_17,Zip_Code_Hash_18,Zip_Code_Hash_19,Claim Injury Type Encoded,Accident_Season_Sin,Accident_Season_Cos
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
5393875,2019-12-30,31.0,NaN,1988.0,0.0,1.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.000000e+00,1.000000e+00
5393091,2019-08-30,46.0,1745.93,1973.0,4.0,4.0,1,1,1,1,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,3,1.224647e-16,-1.000000e+00
5393889,2019-12-06,40.0,1434.80,1979.0,0.0,6.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,3,-1.000000e+00,-1.836970e-16
5393887,2019-12-30,61.0,NaN,1958.0,0.0,1.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.000000e+00,1.000000e+00
5393863,2019-12-26,67.0,NaN,1952.0,0.0,5.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.000000e+00,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6165265,2022-12-14,35.0,NaN,1987.0,0.0,4.0,1,1,1,0,...,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,1,-1.000000e+00,-1.836970e-16
6165285,2022-12-15,33.0,NaN,1989.0,0.0,6.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,-1.000000e+00,-1.836970e-16
6165506,2022-12-13,61.0,991.08,1961.0,1.0,3.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,3,-1.000000e+00,-1.836970e-16


In [3]:
# Import dataset
test_df = pd.read_csv('./preprocessed_data/test_data.csv', index_col = 'Claim Identifier')
test_df

,Accident Date,Age at Injury,Average Weekly Wage,Birth Year,IME-4 Count,Number of Dependents,Known Accident Date,Known Assembly Date,Known C-2 Date,Known C-3 Date,...,Zip_Code_Hash_12,Zip_Code_Hash_13,Zip_Code_Hash_14,Zip_Code_Hash_15,Zip_Code_Hash_16,Zip_Code_Hash_17,Zip_Code_Hash_18,Zip_Code_Hash_19,Accident_Season_Sin,Accident_Season_Cos
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
6165911,2022-12-24,19.0,NaN,2003.0,0.0,1,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,1.000000e+00
6166141,2022-11-20,19.0,NaN,2003.0,0.0,1,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.000000e+00,-1.836970e-16
6165907,2022-12-26,59.0,NaN,1963.0,0.0,0,1,1,1,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000e+00,1.000000e+00
6166047,2022-12-28,55.0,NaN,1967.0,0.0,6,1,1,1,0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,1.000000e+00
6166102,2022-12-20,25.0,NaN,1997.0,0.0,5,1,1,1,0,...,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.000000e+00,-1.836970e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6553137,2012-09-12,52.0,NaN,1960.0,0.0,5,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.224647e-16,-1.000000e+00
6553119,2024-05-22,59.0,NaN,1965.0,0.0,1,1,1,0,1,...,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.000000e+00,6.123234e-17
6553542,2024-05-06,45.0,NaN,1979.0,0.0,5,1,1,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.000000e+00,6.123234e-17


## <span style="color:salmon"> 2. Prepare Dataset </span> 

Define y as a target "Claim Injury Type Encoded" and X with all the other columns

In [4]:
X = df.drop(["Claim Injury Type Encoded"], axis = 1)
y = df["Claim Injury Type Encoded"]

Split the data and the target to X_train, X_test, y_train, y_test, where test_size should be equal to 0.25, stratify equal to target and shuffle equal to True

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.75, stratify = y, shuffle = True)

Define X_test

In [6]:
X_test = test_df.copy()

Check missing values in each dataset:

In [7]:
X_train.isna().sum().sort_values()[X_train.isna().sum() != 0]

Birth Year               538
Age at Injury            538
Average Weekly Wage    91081
dtype: int64

In [8]:
X_val.isna().sum().sort_values()[X_val.isna().sum() != 0]

Birth Year               1648
Age at Injury            1648
Average Weekly Wage    273018
dtype: int64

In [9]:
X_test.isna().sum().sort_values()[X_test.isna().sum() != 0]

Birth Year                982
Age at Injury             982
Average Weekly Wage    335753
dtype: int64

In [10]:
NA_imputer(X_train,X_val)
create_new_features(X_train,X_val)

In [11]:
NA_imputer(X,X_test)
create_new_features(X,X_test)

In [12]:
scaler_train = StandardScaler()
X_train[numerical_features] = scaler_train.fit_transform(X_train[numerical_features])
X_val[numerical_features] = scaler_train.transform(X_val[numerical_features])
scaler_predict = StandardScaler()
X[numerical_features] = scaler_predict.fit_transform(X[numerical_features])
X_test[numerical_features] = scaler_predict.transform(X_test[numerical_features])

## <span style="color:salmon"> 3. XGBoosted </span> 

In [13]:
config = {'max_depth': 5, 'learning_rate': 0.1, 'n_estimators': 100}

In [14]:
model_XGB = XGBClassifier(
        max_depth=config["max_depth"],
        learning_rate=config["learning_rate"],
        n_estimators=config["n_estimators"]
    )
model_XGB.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [15]:
y_train_pred = model_XGB.predict(X_train)

In [16]:
y_val_pred = model_XGB.predict(X_val)

#### <span style="color:salmon"> 3.1  Evaluate the model </span> 


In [17]:
class_mapping = {
    0:'1. CANCELLED', 
    1:'2. NON-COMP',
    2:'3. MED ONLY', 
    3:'4. TEMPORARY',
    4:'5. PPD SCH LOSS', 
    5:'6. PPD NSL', 
    6:'7. PTD', 
    7:'8. DEATH'
}

# Use the values from class_mapping as the target names
target_names = list(class_mapping.values())

Compute confusion matrix to evaluate the accuracy of a classification

In [18]:
# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_train, y_train_pred))
print("\nClassification Report:")
print(classification_report(y_train, y_train_pred, target_names=target_names))

Confusion Matrix:
[[ 1234  1759    49    72     5     0     0     0]
 [  521 71403   315   497    33     0     0     0]
 [   10  9377  1266  5987   586     0     0     0]
 [   10  2131   493 33016  1475     0     0     2]
 [    0   227   176  5711  5956     0     0     0]
 [    0     3    11   898   139     2     0     0]
 [    0     0     0     7     4     0    13     0]
 [    0    15     5    45     2     0     0    51]]

Classification Report:
                 precision    recall  f1-score   support

   1. CANCELLED       0.70      0.40      0.50      3119
    2. NON-COMP       0.84      0.98      0.91     72769
    3. MED ONLY       0.55      0.07      0.13     17226
   4. TEMPORARY       0.71      0.89      0.79     37127
5. PPD SCH LOSS       0.73      0.49      0.59     12070
     6. PPD NSL       1.00      0.00      0.00      1053
         7. PTD       1.00      0.54      0.70        24
       8. DEATH       0.96      0.43      0.60       118

       accuracy                   

In [19]:
# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred, target_names=target_names))


Confusion Matrix:
[[  3386   5587    172    203      8      0      0      0]
 [  1853 213743   1077   1516    106      0      0     14]
 [    55  28142   3092  18409   1977      0      0      5]
 [    44   6721   1647  97875   5067      1      0     25]
 [     5    686    573  17799  17147      0      0      0]
 [     1      9     38   2708    401      1      0      0]
 [     0      0      0     65      8      0      0      0]
 [     2     69     17    210      3      0      0     51]]

Classification Report:


C:\Users\ruipb\Desktop\Masters\Projects\Machine_Learning_NOVAIMS\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                 precision    recall  f1-score   support

   1. CANCELLED       0.63      0.36      0.46      9356
    2. NON-COMP       0.84      0.98      0.90    218309
    3. MED ONLY       0.47      0.06      0.11     51680
   4. TEMPORARY       0.71      0.88      0.78    111380
5. PPD SCH LOSS       0.69      0.47      0.56     36210
     6. PPD NSL       0.50      0.00      0.00      3158
         7. PTD       0.00      0.00      0.00        73
       8. DEATH       0.54      0.14      0.23       352

       accuracy                           0.78    430518
      macro avg       0.55      0.36      0.38    430518
   weighted avg       0.74      0.78      0.73    430518



C:\Users\ruipb\Desktop\Masters\Projects\Machine_Learning_NOVAIMS\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ruipb\Desktop\Masters\Projects\Machine_Learning_NOVAIMS\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## <span style="color:salmon"> 4. Test Predictions </span> 

Make validation predictions:

In [ ]:
break

In [ ]:
y_test_pred = model_XGB.predict(X_test)

In [ ]:
test_id = model_cb.index

In [ ]:
submission_df = pd.DataFrame({
    'Claim Identifier': test_id,
    'Claim Injury Type': y_test_pred
})